In [146]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm 

In [147]:
# Define the CNN model
class SCNN(nn.Sequential):
    def __init__(self):
        super(SCNN, self).__init__(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(64 * 20 * 20, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
class regCNN(nn.Sequential):
    def __init__(self):
        super(regCNN, self).__init__(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(64 * 20 * 20, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )
    


In [148]:
# Device setup for M3 Pro
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS (M3 Pro GPU)")
else:
    device = torch.device("cpu")
    print("Using CPU (MPS not available)")

Using MPS (M3 Pro GPU)


In [161]:
def train(model,num_epochs,patience,train_loader,val_loader,model_path):
    best_val_loss = float('inf')
    patience_counter = 0
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
        for images, labels in train_loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            train_loop.set_postfix(loss=running_loss / (train_loop.n + 1))
        
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.4f}")

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_loader)
        accuracy = 100 * correct / total
        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}% , Best Val Loss: {best_val_loss:.2f}")

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f'{model_path}best.pth')
            # print("Best Model saved successfully!")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered. Training stopped.")
                break
    

In [150]:
def test(model_class ,model_path,val_loader):
    model = model_class
    model.load_state_dict(torch.load(model_path, map_location=device)) 
    model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

# Simple

In [151]:
train_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

# Load Imagenette dataset
qual = '160px'
folder = f'./data/Simple{qual}/'
train_dataset = datasets.Imagenette(root=folder, split='train', size=qual, download=True, transform=train_transform)
val_dataset = datasets.Imagenette(root=folder, split='val', size=qual, download=True, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)



In [152]:
model = SCNN().to(device)
train(model,30,5,train_loader,val_loader,'./model/Simple/')

Epoch [1/30]: 100%|██████████| 148/148 [00:31<00:00,  4.76it/s, loss=1.95]

Epoch [1/30], Training Loss: 1.9468


Validation Loss: 1.6468, Accuracy: 44.69% , Best Val Loss: inf


Epoch [2/30]: 100%|██████████| 148/148 [00:29<00:00,  5.05it/s, loss=1.46]

Epoch [2/30], Training Loss: 1.4294


...

Epoch [8/30], Training Loss: 0.3088


Validation Loss: 1.5728, Accuracy: 61.99% , Best Val Loss: 1.21


Epoch [9/30]: 100%|██████████| 148/148 [00:29<00:00,  5.03it/s, loss=0.198]

Epoch [9/30], Training Loss: 0.1957


Validation Loss: 2.0331, Accuracy: 60.36% , Best Val Loss: 1.21
Early stopping triggered. Training stopped.


In [153]:
test(SCNN(),'./model/Simple/best.pth',val_loader)

Validation Accuracy: 61.02%


# Regularisation

In [154]:
# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Imagenette dataset
qual = '160px'
folder = f'./data/reg{qual}/'
train_dataset_reg = datasets.Imagenette(root=folder, split='train', size=qual, download=True, transform=train_transform)
val_dataset_reg = datasets.Imagenette(root=folder, split='val', size=qual, download=True, transform=val_transform)

train_loader_reg = DataLoader(train_dataset_reg, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader_reg = DataLoader(val_dataset_reg, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)



In [162]:
model2 = regCNN().to(device)

train(model2,30,10,train_loader_reg,val_loader_reg,'./model/Reg/')


Epoch [1/30]: 100%|██████████| 148/148 [00:29<00:00,  4.96it/s, loss=1.96]

Epoch [1/30], Training Loss: 1.9197


Validation Loss: 1.5285, Accuracy: 48.71% , Best Val Loss: inf


Epoch [2/30]: 100%|██████████| 148/148 [00:29<00:00,  4.99it/s, loss=1.53]

Epoch [2/30], Training Loss: 1.4954


...

Epoch [4/30]: 100%|██████████| 148/148 [00:29<00:00,  5.03it/s, loss=1.16]

Epoch [4/30], Training Loss: 1.1570


Validation Loss: 1.0305, Accuracy: 67.77% , Best Val Loss: 1.13


Epoch [5/30]: 100%|██████████| 148/148 [00:29<00:00,  5.06it/s, loss=1.06]

Epoch [5/30], Training Loss: 1.0420


In [156]:
test(regCNN(),'./model/Reg/best.pth',val_loader_reg)

Validation Accuracy: 69.27%


# Transfer Learning

In [157]:
cifar_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])


cifar_train = datasets.CIFAR10(root='./data/cifar/', train=True, download=True, transform=cifar_transform)


cifar_test = datasets.CIFAR10(root='./data/cifar/', train=False, download=True, transform=cifar_transform)


cifar_train_loader = DataLoader(cifar_train, batch_size=64, shuffle=True)
cifar_test_loader = DataLoader(cifar_test, batch_size=64, shuffle=False)

In [158]:
best_model = regCNN().to(device)
best_model.load_state_dict(torch.load("./model/Reg/best.pth"))
for layer in best_model.modules():
    if isinstance(layer, nn.Conv2d):
        for param in layer.parameters():
            param.requires_grad = False

fine_tuned_model = train(best_model,10,3, cifar_train_loader, cifar_test_loader,'./model/cifar/')



Epoch [1/10]: 100%|██████████| 782/782 [00:20<00:00, 38.25it/s, loss=2.08]


Epoch [1/10], Training Loss: 2.0739
Validation Loss: 1.8119, Accuracy: 35.64% , Best Val Loss: inf


Epoch [2/10]: 100%|██████████| 782/782 [00:28<00:00, 27.46it/s, loss=1.91]


Epoch [2/10], Training Loss: 1.9069
Validation Loss: 1.7558, Accuracy: 37.31% , Best Val Loss: 1.81


Epoch [3/10]: 100%|██████████| 782/782 [00:29<00:00, 26.60it/s, loss=1.86]


...

Epoch [8/10], Training Loss: 1.7635
Validation Loss: 1.5511, Accuracy: 45.31% , Best Val Loss: 1.61


Epoch [9/10]: 100%|██████████| 782/782 [00:30<00:00, 25.95it/s, loss=1.75]


Epoch [9/10], Training Loss: 1.7509
Validation Loss: 1.5696, Accuracy: 44.59% , Best Val Loss: 1.55


Epoch [10/10]: 100%|██████████| 782/782 [00:29<00:00, 26.50it/s, loss=1.75]


Epoch [10/10], Training Loss: 1.7455
Validation Loss: 1.5512, Accuracy: 45.23% , Best Val Loss: 1.55


In [159]:
test(regCNN(),'./model/cifar/best.pth',cifar_test_loader)

Validation Accuracy: 45.38%
